In [44]:
import os
os.chdir("D:/tecky-academy/c17-final-project-09-tw/backend/data") 
os.getcwd()

'D:\\tecky-academy\\c17-final-project-09-tw\\backend\\data'

In [45]:
import pandas as pd 
import numpy as np
df = pd.read_csv ('price_1m_20220105.csv')
df.head()

,id,coin,pair,ts_unix,ts_string,open,high,low,close,vol
0,11640707200,BTC,BTC-USD,1640707200,2021-12-29 00:00:00,47623.871094,47623.871094,47623.871094,47623.871094,0
1,11640707260,BTC,BTC-USD,1640707260,2021-12-29 00:01:00,47642.886719,47642.886719,47642.886719,47642.886719,0
2,11640707320,BTC,BTC-USD,1640707320,2021-12-29 00:02:00,47660.617188,47660.617188,47660.617188,47660.617188,5519360
3,11640707380,BTC,BTC-USD,1640707380,2021-12-29 00:03:00,47625.851562,47625.851562,47625.851562,47625.851562,0
4,11640707440,BTC,BTC-USD,1640707440,2021-12-29 00:04:00,47576.855469,47576.855469,47576.855469,47576.855469,0


In [46]:
import time
from datetime import datetime 
start = min(df['ts_unix'])
print(start)
print(datetime.fromtimestamp(start))
end = max(df['ts_unix'])
print(end)
print(datetime.fromtimestamp(end))
elapsed = (end-start)/60
print(elapsed)

1640707200
2021-12-29 00:00:00
1641248160
2022-01-04 06:16:00
9016.0


In [47]:
import random
# random.seed(12345)
print(random.randint(start, end))

1640868389


In [48]:
num_transaction = 5000
col_names = ["side","pair","executedPrice","executed","amount","fee","unixTimestamp","datetime"]
transaction_seed_df = pd.DataFrame(index=np.arange(num_transaction), columns=col_names)
transaction_seed_df.head()

,side,pair,executedPrice,executed,amount,fee,unixTimestamp,datetime
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
np.unique(df['pair'])

array(['BTC-USD', 'ETH-USD', 'SAND-USD'], dtype=object)

In [50]:
for i in range(num_transaction):
    transaction_seed_df['unixTimestamp'][i] = start + random.randint(0, elapsed)*60
    transaction_seed_df['datetime'][i] = datetime.fromtimestamp(transaction_seed_df['unixTimestamp'][i])
    # generate sides
    aux = random.randint(0,1)
    if aux == 0:
        transaction_seed_df['side'][i] = "buy"
    elif aux == 1:
        transaction_seed_df['side'][i] = "sell"
    # generate pair
    aux2 = random.randint(0,2)
    if aux2 == 0:
        transaction_seed_df['pair'][i] = "BTC-USD"
    elif aux2 == 1:
        transaction_seed_df['pair'][i] = "ETH-USD"
    elif aux2 == 2:
        transaction_seed_df['pair'][i] = "SAND-USD"
transaction_seed_df.sort_values(by='unixTimestamp', inplace=True, ascending=True, ignore_index=True)  
transaction_seed_df.head()

,side,pair,executedPrice,executed,amount,fee,unixTimestamp,datetime
0,buy,SAND-USD,NaN,NaN,NaN,NaN,1640707320,2021-12-29 00:02:00
1,sell,ETH-USD,NaN,NaN,NaN,NaN,1640707380,2021-12-29 00:03:00
2,buy,BTC-USD,NaN,NaN,NaN,NaN,1640707380,2021-12-29 00:03:00
3,buy,ETH-USD,NaN,NaN,NaN,NaN,1640707620,2021-12-29 00:07:00
4,buy,ETH-USD,NaN,NaN,NaN,NaN,1640707620,2021-12-29 00:07:00


In [51]:
# get executed price from df and post to transaction_seed_df
print(len(transaction_seed_df))
merge_item = []

for i in range(len(transaction_seed_df)):
    temp = str(transaction_seed_df["pair"][i]) + str(transaction_seed_df["unixTimestamp"][i])
    merge_item.append(temp)
transaction_seed_df["merge_item"] = merge_item
print(type(merge_item[0]))

transaction_seed_df.head()

merge_item = []

for i in range(len(df)):
    temp = str(df["pair"][i]) + str(df["ts_unix"][i])
    merge_item.append(temp)
df["merge_item"] = merge_item

df.head()

temp_df = df[['close','merge_item']]
temp_df.head()
# transaction_seed_df = pd.merge(transaction_seed_df, df["close","merge_item"], on="merge_item", how="inner")
temp_df2 = pd.merge(transaction_seed_df, temp_df, on="merge_item", how="inner")
for i in range(len(temp_df2)):
    temp_df2["executedPrice"][i] = temp_df2["close"][i]

# trim unused columns
transaction_seed_df = temp_df2

transaction_seed_df.drop(["close","merge_item"], axis=1, inplace = True)

transaction_seed_df.head()

5000
<class 'str'>


C:\Users\wtklau\AppData\Local\Temp/ipykernel_14712/213613314.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df2["executedPrice"][i] = temp_df2["close"][i]


,side,pair,executedPrice,executed,amount,fee,unixTimestamp,datetime
0,buy,SAND-USD,5.880234,NaN,NaN,NaN,1640707320,2021-12-29 00:02:00
1,sell,ETH-USD,3801.717773,NaN,NaN,NaN,1640707380,2021-12-29 00:03:00
2,buy,BTC-USD,47625.851562,NaN,NaN,NaN,1640707380,2021-12-29 00:03:00
3,buy,ETH-USD,3805.050049,NaN,NaN,NaN,1640707620,2021-12-29 00:07:00
4,buy,ETH-USD,3805.050049,NaN,NaN,NaN,1640707620,2021-12-29 00:07:00


In [52]:
# insert initial capital
transaction_seed_df.loc[-1] = ["buy","USD-USD",1,1000000,1000000,0,1640707200,"2021-12-29 00:00:00"]
transaction_seed_df.index = transaction_seed_df.index + 1  # shifting index
transaction_seed_df = transaction_seed_df.sort_index()
transaction_seed_df.head()

,side,pair,executedPrice,executed,amount,fee,unixTimestamp,datetime
0,buy,USD-USD,1,1000000,1000000,0,1640707200,2021-12-29 00:00:00
1,buy,SAND-USD,5.880234,NaN,NaN,NaN,1640707320,2021-12-29 00:02:00
2,sell,ETH-USD,3801.717773,NaN,NaN,NaN,1640707380,2021-12-29 00:03:00
3,buy,BTC-USD,47625.851562,NaN,NaN,NaN,1640707380,2021-12-29 00:03:00
4,buy,ETH-USD,3805.050049,NaN,NaN,NaN,1640707620,2021-12-29 00:07:00


In [53]:
# fill executed (get running sum amount for the four assets, need to add new columns)
import math
zero_vector = []
for i in range(len(transaction_seed_df)):
    zero_vector.append(0.00000000000000)

transaction_seed_df["BTC"] = zero_vector
transaction_seed_df["ETH"] = zero_vector
transaction_seed_df["SAND"] = zero_vector
transaction_seed_df["USD"] = zero_vector

transaction_seed_df["USD"][0] = transaction_seed_df["executed"][0]

adValFee = 0.001
for i in range(1,len(transaction_seed_df)):
    if transaction_seed_df["side"][i] == "buy":
        
        max_execute = math.floor(transaction_seed_df["USD"][i-1] / (1 + adValFee) / transaction_seed_df["executedPrice"][i])
        transaction_seed_df["executed"][i] = random.random() * max_execute
        transaction_seed_df["amount"][i] = transaction_seed_df["executed"][i] * transaction_seed_df["executedPrice"][i]
        
        transaction_seed_df["USD"][i] = transaction_seed_df["USD"][i-1] - transaction_seed_df["amount"][i] * (1 + adValFee)
        
        if transaction_seed_df["pair"][i] == "BTC-USD":
            transaction_seed_df["BTC"][i] = transaction_seed_df["BTC"][i-1] + transaction_seed_df["executed"][i]
            transaction_seed_df["ETH"][i] = transaction_seed_df["ETH"][i-1]
            transaction_seed_df["SAND"][i] = transaction_seed_df["SAND"][i-1]
        elif transaction_seed_df["pair"][i] == "ETH-USD":
            transaction_seed_df["BTC"][i] = transaction_seed_df["BTC"][i-1]
            transaction_seed_df["ETH"][i] = transaction_seed_df["ETH"][i-1] + transaction_seed_df["executed"][i]
            transaction_seed_df["SAND"][i] = transaction_seed_df["SAND"][i-1]
        elif transaction_seed_df["pair"][i] == "SAND-USD":
            transaction_seed_df["BTC"][i] = transaction_seed_df["BTC"][i-1]
            transaction_seed_df["ETH"][i] = transaction_seed_df["ETH"][i-1]
            transaction_seed_df["SAND"][i] = transaction_seed_df["SAND"][i-1] + transaction_seed_df["executed"][i]
    
    elif transaction_seed_df["side"][i] == "sell":

        if transaction_seed_df["pair"][i] == "BTC-USD":
            max_execute = transaction_seed_df["BTC"][i-1]
        elif transaction_seed_df["pair"][i] == "ETH-USD":
            max_execute = transaction_seed_df["ETH"][i-1]
        elif transaction_seed_df["pair"][i] == "SAND-USD":
            max_execute = transaction_seed_df["SAND"][i-1]

        transaction_seed_df["executed"][i] = random.random() * max_execute
        transaction_seed_df["amount"][i] = transaction_seed_df["executed"][i] * transaction_seed_df["executedPrice"][i]
        
        transaction_seed_df["USD"][i] = transaction_seed_df["USD"][i-1] + transaction_seed_df["amount"][i] * (1 - adValFee)
        
        if transaction_seed_df["pair"][i] == "BTC-USD":
            transaction_seed_df["BTC"][i] = transaction_seed_df["BTC"][i-1] - transaction_seed_df["executed"][i]
            transaction_seed_df["ETH"][i] = transaction_seed_df["ETH"][i-1]
            transaction_seed_df["SAND"][i] = transaction_seed_df["SAND"][i-1]
        elif transaction_seed_df["pair"][i] == "ETH-USD":
            transaction_seed_df["BTC"][i] = transaction_seed_df["BTC"][i-1]
            transaction_seed_df["ETH"][i] = transaction_seed_df["ETH"][i-1] - transaction_seed_df["executed"][i]
            transaction_seed_df["SAND"][i] = transaction_seed_df["SAND"][i-1]
        elif transaction_seed_df["pair"][i] == "SAND-USD":
            transaction_seed_df["BTC"][i] = transaction_seed_df["BTC"][i-1]
            transaction_seed_df["ETH"][i] = transaction_seed_df["ETH"][i-1]
            transaction_seed_df["SAND"][i] = transaction_seed_df["SAND"][i-1] - transaction_seed_df["executed"][i]

transaction_seed_df.head()

C:\Users\wtklau\AppData\Local\Temp/ipykernel_14712/3640528610.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_seed_df["USD"][0] = transaction_seed_df["executed"][0]
C:\Users\wtklau\AppData\Local\Temp/ipykernel_14712/3640528610.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_seed_df["executed"][i] = random.random() * max_execute
C:\Users\wtklau\AppData\Local\Temp/ipykernel_14712/3640528610.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

,side,pair,executedPrice,executed,amount,fee,unixTimestamp,datetime,BTC,ETH,SAND,USD
0,buy,USD-USD,1,1000000,1000000,0,1640707200,2021-12-29 00:00:00,0.000000,0.000000,0.00000,1000000.000000
1,buy,SAND-USD,5.880234,41750.02834,245499.946229,NaN,1640707320,2021-12-29 00:02:00,0.000000,0.000000,41750.02834,754254.553825
2,sell,ETH-USD,3801.717773,0.0,0.0,NaN,1640707380,2021-12-29 00:03:00,0.000000,0.000000,41750.02834,754254.553825
3,buy,BTC-USD,47625.851562,6.455815,307463.692509,NaN,1640707380,2021-12-29 00:03:00,6.455815,0.000000,41750.02834,446483.397624
4,buy,ETH-USD,3805.050049,70.443669,268041.684992,NaN,1640707620,2021-12-29 00:07:00,6.455815,70.443669,41750.02834,178173.670946


In [54]:
# get fee and amount
for i in range(len(transaction_seed_df)):
    if transaction_seed_df["pair"][i] != "USD-USD":
        transaction_seed_df["amount"][i] = transaction_seed_df["executedPrice"][i] * transaction_seed_df["executed"][i]
        transaction_seed_df["fee"][i] = transaction_seed_df["amount"][i] * adValFee

transaction_seed_df.head()

C:\Users\wtklau\AppData\Local\Temp/ipykernel_14712/4084298453.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_seed_df["amount"][i] = transaction_seed_df["executedPrice"][i] * transaction_seed_df["executed"][i]
C:\Users\wtklau\AppData\Local\Temp/ipykernel_14712/4084298453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_seed_df["fee"][i] = transaction_seed_df["amount"][i] * adValFee


,side,pair,executedPrice,executed,amount,fee,unixTimestamp,datetime,BTC,ETH,SAND,USD
0,buy,USD-USD,1,1000000,1000000,0,1640707200,2021-12-29 00:00:00,0.000000,0.000000,0.00000,1000000.000000
1,buy,SAND-USD,5.880234,41750.02834,245499.946229,245.499946,1640707320,2021-12-29 00:02:00,0.000000,0.000000,41750.02834,754254.553825
2,sell,ETH-USD,3801.717773,0.0,0.0,0.0,1640707380,2021-12-29 00:03:00,0.000000,0.000000,41750.02834,754254.553825
3,buy,BTC-USD,47625.851562,6.455815,307463.692509,307.463693,1640707380,2021-12-29 00:03:00,6.455815,0.000000,41750.02834,446483.397624
4,buy,ETH-USD,3805.050049,70.443669,268041.684992,268.041685,1640707620,2021-12-29 00:07:00,6.455815,70.443669,41750.02834,178173.670946


In [55]:
# check and drop auxillary BTC, ETH, SAND, USD
check = []
for i in range(len(transaction_seed_df)):
    if transaction_seed_df['BTC'][i] < 0:
        check.append(1)
    if transaction_seed_df['ETH'][i] < 0:
        check.append(1)
    if transaction_seed_df['SAND'][i] < 0:
        check.append(1)
    if transaction_seed_df['USD'][i] < 0:
        check.append(1)
check_sum = sum(check)
print(check_sum)

# transaction_seed_df.drop(["BTC","ETH","SAND","USD"], axis=1, inplace = True)
transaction_seed_df.head()

0


,side,pair,executedPrice,executed,amount,fee,unixTimestamp,datetime,BTC,ETH,SAND,USD
0,buy,USD-USD,1,1000000,1000000,0,1640707200,2021-12-29 00:00:00,0.000000,0.000000,0.00000,1000000.000000
1,buy,SAND-USD,5.880234,41750.02834,245499.946229,245.499946,1640707320,2021-12-29 00:02:00,0.000000,0.000000,41750.02834,754254.553825
2,sell,ETH-USD,3801.717773,0.0,0.0,0.0,1640707380,2021-12-29 00:03:00,0.000000,0.000000,41750.02834,754254.553825
3,buy,BTC-USD,47625.851562,6.455815,307463.692509,307.463693,1640707380,2021-12-29 00:03:00,6.455815,0.000000,41750.02834,446483.397624
4,buy,ETH-USD,3805.050049,70.443669,268041.684992,268.041685,1640707620,2021-12-29 00:07:00,6.455815,70.443669,41750.02834,178173.670946


In [56]:
# drop entries with executed being zero
drop_index = []
print(len(transaction_seed_df))
for i in range(1,len(transaction_seed_df)):
    if transaction_seed_df['executed'][i] == 0:
        drop_index.append(i)
transaction_seed_df.drop(drop_index, axis=0, inplace=True)

# new_index = []
# print(len(transaction_seed_df))
# for j in range(len(transaction_seed_df)):
#     new_index.append(j)
# print(new_index)
# transaction_seed_df.reindex(new_index)

transaction_seed_df.reset_index(drop=True, inplace=True)
transaction_seed_df.head()

4394


,side,pair,executedPrice,executed,amount,fee,unixTimestamp,datetime,BTC,ETH,SAND,USD
0,buy,USD-USD,1,1000000,1000000,0,1640707200,2021-12-29 00:00:00,0.000000,0.000000,0.00000,1000000.000000
1,buy,SAND-USD,5.880234,41750.02834,245499.946229,245.499946,1640707320,2021-12-29 00:02:00,0.000000,0.000000,41750.02834,754254.553825
2,buy,BTC-USD,47625.851562,6.455815,307463.692509,307.463693,1640707380,2021-12-29 00:03:00,6.455815,0.000000,41750.02834,446483.397624
3,buy,ETH-USD,3805.050049,70.443669,268041.684992,268.041685,1640707620,2021-12-29 00:07:00,6.455815,70.443669,41750.02834,178173.670946
4,buy,ETH-USD,3805.050049,44.33193,168685.213283,168.685213,1640707620,2021-12-29 00:07:00,6.455815,114.775599,41750.02834,9319.772450


In [57]:
# insert USD entries
for i in range(1,len(transaction_seed_df)):
    if transaction_seed_df["side"][i] == "buy":
        temp_df = pd.DataFrame({'side': ['sell','sell'],
                    'pair' : ['USD-USD','USD-USD'],
                    'executedPrice' : [1,1],
                    'executed' : [transaction_seed_df["fee"][i],transaction_seed_df["amount"][i]],
                    'amount' : [transaction_seed_df["fee"][i],transaction_seed_df["amount"][i]],
                    'fee' : [0,0],
                    'unixTimestamp' : [transaction_seed_df["unixTimestamp"][i],transaction_seed_df["unixTimestamp"][i]],
                    'datetime' : [transaction_seed_df["datetime"][i],transaction_seed_df["datetime"][i]],
                    'BTC': [transaction_seed_df["BTC"][i],transaction_seed_df["BTC"][i]],
                    'ETH': [transaction_seed_df["ETH"][i],transaction_seed_df["ETH"][i]],
                    'SAND': [transaction_seed_df["SAND"][i],transaction_seed_df["SAND"][i]],
                    'USD': [transaction_seed_df["USD"][i],transaction_seed_df["USD"][i]]
                    })
    
    elif transaction_seed_df["side"][i] == "sell":
        temp_df = pd.DataFrame({'side': ['sell','buy'],
                    'pair' : ['USD-USD','USD-USD'],
                    'executedPrice' : [1,1],
                    'executed' : [transaction_seed_df["fee"][i],transaction_seed_df["amount"][i]],
                    'amount' : [transaction_seed_df["fee"][i],transaction_seed_df["amount"][i]],
                    'fee' : [0,0],
                    'unixTimestamp' : [transaction_seed_df["unixTimestamp"][i],transaction_seed_df["unixTimestamp"][i]],
                    'datetime' : [transaction_seed_df["datetime"][i],transaction_seed_df["datetime"][i]],
                    'BTC': [transaction_seed_df["BTC"][i],transaction_seed_df["BTC"][i]],
                    'ETH': [transaction_seed_df["ETH"][i],transaction_seed_df["ETH"][i]],
                    'SAND': [transaction_seed_df["SAND"][i],transaction_seed_df["SAND"][i]],
                    'USD': [transaction_seed_df["USD"][i],transaction_seed_df["USD"][i]]
                    })
    transaction_seed_df = pd.concat([transaction_seed_df, temp_df], axis = 0)

transaction_seed_df.sort_values(by='unixTimestamp', inplace=True, ascending=True, ignore_index=True)  
transaction_seed_df.head()



,side,pair,executedPrice,executed,amount,fee,unixTimestamp,datetime,BTC,ETH,SAND,USD
0,buy,USD-USD,1,1000000,1000000,0,1640707200,2021-12-29 00:00:00,0.000000,0.0,0.00000,1000000.000000
1,sell,USD-USD,1,245499.946229,245499.946229,0,1640707320,2021-12-29 00:02:00,0.000000,0.0,41750.02834,754254.553825
2,sell,USD-USD,1,245.499946,245.499946,0,1640707320,2021-12-29 00:02:00,0.000000,0.0,41750.02834,754254.553825
3,buy,SAND-USD,5.880234,41750.02834,245499.946229,245.499946,1640707320,2021-12-29 00:02:00,0.000000,0.0,41750.02834,754254.553825
4,sell,USD-USD,1,307.463693,307.463693,0,1640707380,2021-12-29 00:03:00,6.455815,0.0,41750.02834,446483.397624


In [58]:
# export to csv
workdir = os.getcwd()
write_to_path = os.path.join(workdir, "transaction-seed.csv")
transaction_seed_df.to_csv(path_or_buf = write_to_path, index = False)